# Word Transcript Tabulator Recipe

This notebook is a computational recipe to help you to take a collection of transcripts in word documents (_docx_ only) and turn them into a structured tabular format suitable for working with computational text analytics.

As a side-effect of this, it will also help you check for and identify some inconsistencies in your transcripts (for example, inconsistent speaker identifiers).

Note that this approach will remove all formatting from your text: if you have information in *bold*, _italics_, or elements highlighted in different colours that information will be lost in this process.

## Workflow Overview

1. Upload your transcripts in `.docx` format to the transcripts folder.
2. Run the script to produce an initial file for examination and manual changes.
3. Check that the headers have been correctly been identified. (How???)
4. Use the produced metadata to identify and fix inconsistencies in the transcripts *in word* - reupload any changed versions and try again.
5. Fill in the metadata for speaker_codes and speakers. (Match codes in transcripts to speaker_ids, give details known about speakers).

(This is going to need screenshots...)

## Data Model

## Libraries We'll Use

## TODO: how to do this iteratively?

## USECASE: while you're in data collection stage!

# Getting Started

1. Upload your transcripts in `.docx` format in the transcripts folder on the left. Nested files are currently not supported.
2. Run the code below.


## TODO: Breakdown the functions and explain them one by one.

In [43]:
library(officer)
library(writexl)
library(dplyr)
library(stringr)

# Define some helper functions: these are how we extract individual units of data from your transcripts

# Try to guess where a transcript might start after any header information - this can be fixed in the next step.
guess_header_rows <- function(doc_summary){
    # Go through all of the lines 1 by 1 until they match one of the potential ends of the header block
    for (i in 1:nrow(doc_summary)){
        row = doc_summary[i,]
        # Header might end with a first blank line
        if (trimws(row$text) == '') {
            break
        }
        # Header ends with first list paragraph style (ie, if you use numbered lists to generate turn numbers)
        else if (!is.na(row$style_name) & row$style_name == 'List Paragraph'){
            break
        }
    }
    
    # If we can't find an end of header match, mark nothing as the header.
    if (i == nrow(doc_summary)){
        i <- 1
    }
    
    doc_summary$header = FALSE
    doc_summary$header[1:i] = TRUE
    # If we don't match any other rules, set the header to be zero lines long
    # at the top of the file.
    return(doc_summary)
}


# TODO: extract any timecodes

# Do the initial extraction from a particular file
prepare_transcripts <- function(transcript_file_path){
    print(transcript_file_path)
    transcript_df <- officer::docx_summary(officer::read_docx(transcript_file_path))
    
    # Attach the filepath as a column so we can trace this back
    transcript_df$source_file <- basename(transcript_file_path)
    
    # Keep a copy of the original text so we can always compare the processing
    transcript_df$source_text <- transcript_df$text
    
    # Extract the speaker codes via regex - match up to, but don't include the first colon.
    speaker_match = '^[[:alpha:]]+?(?=:)'
    transcript_df$speaker <- stringr::str_extract(transcript_df$source_text, speaker_match)
    
    # And replace the original text with the speaker code removed
    transcript_df$text <- NULL
    transcript_df$text <- str_replace(transcript_df$source_text, '^[[:alpha:]]+?:', "")
    
    return(transcript_df)
}

In [47]:
# Now let's actually load your transcripts

# List all of the docx files in the transcripts folder
transcript_files = list.files("transcripts", full.names=TRUE, pattern="*.docx")

paste0("Loading: ", length(transcript_files), " transcripts:") 
transcript_files

# TODO: figure out how to handle warnings without scaring people and ignoring them...
# Load each transcript, extract the paragraphs of the document as rows in a dataframe, using the officer package
loaded_docs <- lapply(transcript_files, prepare_transcripts)

# Combine the transcripts together
combined_transcripts <- bind_rows(loaded_docs)


[1] "Loading:23"

[1] "transcripts/AmAus01_transcript.docx" "transcripts/AmAus02_transcript.docx"
 [3] "transcripts/AmAus03_transcript.docx" "transcripts/AmAus04_transcript.docx"
 [5] "transcripts/AmAus05_transcript.docx" "transcripts/AmAus06_transcript.docx"
 [7] "transcripts/AmAus07_transcript.docx" "transcripts/AmAus08_transcript.docx"
 [9] "transcripts/AmAus09_transcript.docx" "transcripts/AmAus10_transcript.docx"
[11] "transcripts/AmAus11_transcript.docx" "transcripts/AmAus12_transcript.docx"
[13] "transcripts/AmAus13_transcript.docx" "transcripts/AmAus14_transcript.docx"
[15] "transcripts/AmAus15_transcript.docx" "transcripts/AmAus16_transcript.docx"
[17] "transcripts/AmAus17_transcript.docx" "transcripts/AmAus18_transcript.docx"
[19] "transcripts/AmAus19_transcript.docx" "transcripts/AmAus20_transcript.docx"
[21] "transcripts/AmAus21_transcript.docx" "transcripts/AmAus22_transcript.docx"
[23] "transcripts/AmAus23_transcript.docx"

[1] "transcripts/AmAus01_transcript.docx"
[1] "transcripts/AmAus02_transcript.docx"


Warning message in xml_find_all.xml_node(doc, "/cp:coreProperties/*"):
“Undefined namespace prefix [1219]”


[1] "transcripts/AmAus03_transcript.docx"
[1] "transcripts/AmAus04_transcript.docx"
[1] "transcripts/AmAus05_transcript.docx"
[1] "transcripts/AmAus06_transcript.docx"


...

[1] "transcripts/AmAus07_transcript.docx"
[1] "transcripts/AmAus08_transcript.docx"
[1] "transcripts/AmAus09_transcript.docx"
[1] "transcripts/AmAus10_transcript.docx"


Warning message in xml_find_all.xml_node(doc, "/cp:coreProperties/*"):
“Undefined namespace prefix [1219]”


[1] "transcripts/AmAus11_transcript.docx"
[1] "transcripts/AmAus12_transcript.docx"
[1] "transcripts/AmAus13_transcript.docx"
[1] "transcripts/AmAus14_transcript.docx"
[1] "transcripts/AmAus15_transcript.docx"
[1] "transcripts/AmAus16_transcript.docx"
[1] "transcripts/AmAus17_transcript.docx"
[1] "transcripts/AmAus18_transcript.docx"
[1] "transcripts/AmAus19_transcript.docx"


Warning message in xml_find_all.xml_node(doc, "/cp:coreProperties/*"):
“Undefined namespace prefix [1219]”


[1] "transcripts/AmAus20_transcript.docx"
[1] "transcripts/AmAus21_transcript.docx"
[1] "transcripts/AmAus22_transcript.docx"
[1] "transcripts/AmAus23_transcript.docx"


In [45]:
combined_transcripts %>% select(source_file, speaker) %>% filter(!is.na(speaker)) %>% unique() %>% count()

n
<int>
104


In [37]:
combined_transcripts$text2 <- str_replace(combined_transcripts$text, '^[[:alpha:]]+?:', "")

In [46]:
combined_transcripts

doc_index,content_type,style_name,level,num_id,source_file,source_text,speaker,text
<int>,<chr>,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>
1,paragraph,NA,NA,NA,AmAus01_transcript.docx,Transcript – AmAus01_transcript,NA,Transcript – AmAus01_transcript
2,paragraph,NA,NA,NA,AmAus01_transcript.docx,Recording date: 10 August 2012,NA,Recording date: 10 August 2012
3,paragraph,NA,NA,NA,AmAus01_transcript.docx,Length of recording:,NA,Length of recording:
4,paragraph,NA,NA,NA,AmAus01_transcript.docx,Speakers:,Speakers,
5,paragraph,NA,NA,NA,AmAus01_transcript.docx,A:Chuck (American),A,Chuck (American)
6,paragraph,NA,NA,NA,AmAus01_transcript.docx,B:Maelee (Australian),B,Maelee (Australian)
7,paragraph,NA,NA,NA,AmAus01_transcript.docx,C:Lara,C,Lara
8,paragraph,NA,NA,NA,AmAus01_transcript.docx,,NA,
9,paragraph,List Paragraph,NA,NA,AmAus01_transcript.docx,@ ~ 1:01,NA,@ ~ 1:01
